# Data exploration video

In [91]:
import os
import sys
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

import torch
import torchvision

# To add src to the path so that we can import modules
current_directory = os.getcwd()
if not current_directory.endswith("emotion_recognition"):
    sys.path.append(os.path.join(current_directory, 'emotion_recognition'))

try:
    from src import TEST_VIDEO_DIR
except ModuleNotFoundError:
    print("Ensure that src is added to PATH and restart the kernel")
    print(sys.path)

I want to convert this xml file in a pandas dataset that for each same filename saves all frames as a key and the value is the list of bboxes and label assigned

In [92]:
annotations_path = os.path.join(TEST_VIDEO_DIR, 'output.xml')
# Parsear el archivo XML
tree = ET.parse(annotations_path)
root = tree.getroot()

data = []

# Iterar sobre cada elemento 'image'
for image in root.iter('image'):
    id = image.get('name')
    filename = id.split("/")[0]
    frame = int(id.split("/")[1].split(".")[0])
    width = int(image.get('width'))
    height = int(image.get('height'))

    boxes = list(image.iter('box'))
    if not boxes:
        print(f"The frame {id.split('.')[0]} has no bbox")
        label = -1
        bbox = np.array([-1, -1, -1, -1])
        
        # Añadir los datos a la lista
        data.append([filename, frame, label, bbox])
        continue

    # Iterar sobre cada elemento 'box'
    for box in boxes:
        label = int(box.get('label').split(":")[0])
        xtl = max(int(float(box.get('xtl'))), 0)
        ytl = max(int(float(box.get('ytl'))), 0)
        xbr = min(int(float(box.get('xbr'))), width)
        ybr = min(int(float(box.get('ybr'))), height)
        bbox = np.array([xtl, ytl, xbr, ybr])
        
        # Añadir los datos a la lista
        data.append([filename, frame, label, bbox])

The frame no_audio_gopro_recording1/0000387 has no bbox
The frame no_audio_gopro_recording1/0000388 has no bbox
The frame no_audio_gopro_recording1/0000389 has no bbox
The frame no_audio_gopro_recording1/0000390 has no bbox
The frame no_audio_gopro_recording1/0000391 has no bbox
The frame no_audio_gopro_recording1/0000392 has no bbox
The frame no_audio_gopro_recording1/0000393 has no bbox
The frame no_audio_gopro_recording1/0000394 has no bbox
The frame no_audio_gopro_recording1/0000395 has no bbox
The frame no_audio_gopro_recording1/0000396 has no bbox
The frame no_audio_gopro_recording1/0000397 has no bbox
The frame no_audio_gopro_recording1/0000398 has no bbox
The frame no_audio_gopro_recording1/0000399 has no bbox
The frame no_audio_gopro_recording1/0000400 has no bbox
The frame no_audio_gopro_recording1/0000401 has no bbox
The frame no_audio_gopro_recording1/0000402 has no bbox
The frame no_audio_gopro_recording1/0000403 has no bbox
The frame no_audio_gopro_recording1/0000404 has 

In [99]:
# Convertir la lista en un DataFrame
df = pd.DataFrame(data, columns=['filename', 'frame', 'label', 'bbox'])

# Crear un nuevo DataFrame agrupando por nombre de archivo y fotograma, y convertir cada grupo en una matriz 2D de numpy para 'bboxes' y una matriz 1D para 'labels'
df_grouped = df.groupby(['filename', 'frame']).apply(lambda g: pd.DataFrame({
    'bboxes': [torch.tensor(np.vstack(g['bbox'].values))],
    'labels': [torch.Tensor(g['label'].values)]}
    )).reset_index()


# Delete unused columns
df_grouped = df_grouped.drop(columns=['level_2'])
filtered_df = df_grouped[df_grouped['filename'] == 'no_audio_gopro_recording1']


print(filtered_df.iloc[1000]['labels'])

tensor([-1.])


In [102]:
print(df_grouped.loc[4586]['bboxes'])

print(df_grouped.loc[4586]['labels'])
print(type(df_grouped.loc[4586]['labels']))
print(df_grouped.loc[4586])

tensor([[1177,  271, 1329,  482],
        [ 576,  324,  747,  534],
        [ 578,  324,  745,  527],
        [ 940,  380, 1017,  469]])
tensor([0., 0., 0., 0.])
<class 'torch.Tensor'>
filename                            no_audio_gopro_recording1
frame                                                       0
bboxes      [[tensor(1177), tensor(271), tensor(1329), ten...
labels       [tensor(0.), tensor(0.), tensor(0.), tensor(0.)]
Name: 4586, dtype: object


In [95]:
# Specify the name and frame you are looking for
target_name = "izhar_recording2"
target_frame = 4585

# Iterate through the 'data' list to find the matching entry
for entry in data:
    if entry[0] == target_name and entry[1] == target_frame:
        print("Found matching entry:")
        print(entry)
        break

Found matching entry:
['izhar_recording2', 4585, 0, array([ 126,  693,  460, 1168])]


In [96]:
GT_bboxes = torch.tensor(df_grouped.loc[4586]['bboxes'])
faces_bbox = torch.tensor(df_grouped.loc[4586]['bboxes'])
GT_bboxes = torch.cat((GT_bboxes, torch.tensor([[0, 0, 100, 100]])), dim=0)


print(torchvision.ops.box_iou(GT_bboxes, faces_bbox))
value, id = torch.max(torchvision.ops.box_iou(GT_bboxes, faces_bbox), dim = 1)
print(value, id)
print(value > 0.5)

/tmp/ipykernel_3593814/2120006449.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  GT_bboxes = torch.tensor(df_grouped.loc[4586]['bboxes'])
/tmp/ipykernel_3593814/2120006449.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  faces_bbox = torch.tensor(df_grouped.loc[4586]['bboxes'])


RuntimeError: Tensors must have same number of dimensions: got 1 and 2